**This is a XGBoost only with numeric features**

# Libraries

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
data_train = pd.read_csv('https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_1_nacho/intento_1/train_no_duplicates_with_word_classes.csv',\
                         sep=',', usecols = ['url_count',
       'hashtag_count', 'mention_count', 'digits_count', 'characters_count',
       'characters_count_clean', 'characters_count_clean_sw', 'word_count',
       'word_count_clean', 'word_count_clean_sw', 'avg_word_len',
       'avg_word_len_clean_sw', 'PRP$', 'NNS', 'VBP', 'DT', 'NN', 'IN', 'MD',
       'VB', 'JJ', 'PRP', 'JJS', 'VBD', 'TO', 'VBG', 'VBN', 'CC', 'CD', 'RB',
       'EX', 'VBZ', 'WP', 'RP', 'JJR', 'WRB', '$', 'WDT', 'NNP', 'RBR', 'RBS',
       'PDT', 'SYM', 'FW', 'UH', 'X', 'WP$', 'target'],\
       dtype = {'PRP$': 'int64', 'NNS': 'int64', 'VBP': 'int64', 'DT': 'int64', 'NN': 'int64', 'IN': 'int64', 'MD': 'int64',
       'VB': 'int64', 'JJ': 'int64', 'PRP': 'int64', 'JJS': 'int64', 'VBD': 'int64', 'TO': 'int64', 'VBG': 'int64', 'VBN': 'int64',\
       'CC': 'int64', 'CD': 'int64', 'RB': 'int64','EX': 'int64', 'VBZ': 'int64', 'WP': 'int64', 'RP': 'int64', 'JJR': 'int64',\
       'WRB': 'int64', '$': 'int64', 'WDT': 'int64', 'NNP': 'int64', 'RBR': 'int64', 'RBS': 'int64','PDT': 'int64', 'SYM': 'int64',\
       'FW': 'int64', 'UH': 'int64', 'X': 'int64', 'WP$': 'int64', 'target': 'int64'})



data_test = pd.read_csv('https://raw.githubusercontent.com/Ignacio-Ibarra/NLP-Disasters/intento_1_nacho/intento_1/featured_test.csv',sep=',')
data_test = data_test[data_test.columns[1:]]
data_test.columns = ['id', 'keyword', 'location', 'text', 'text_clean', 'text_clean_no_sw',
       'url_count', 'hashtag_count', 'mention_count', 'digits_count',
       'characters_count', 'characters_count_clean',
       'characters_count_clean_sw', 'word_count', 'word_count_clean',
       'word_count_clean_sw', 'avg_word_len', 'avg_word_len_clean_sw',
       'tokenized_text', 'pos_tagged_text', 'pos_tagg_counts', 'RB', 'VBD',
       'DT', 'JJ', 'NN', 'IN', 'VBZ', 'NNS', 'VBP', 'EX', 'VBG', 'VB', 'PRP',
       'CD', 'CC', 'MD', 'RBR', 'WRB', 'WP', 'VBN', 'RP', 'PRP$', '$', 'TO',
       'NNP', 'WDT', 'PDT', 'JJS', 'JJR', 'FW', 'RBS', 'X', 'UH']
id = list(data_test.id)
x_test_numeric = data_test[['url_count', 'hashtag_count', 'mention_count', 'digits_count',
       'characters_count', 'characters_count_clean',
       'characters_count_clean_sw', 'word_count', 'word_count_clean',
       'word_count_clean_sw', 'avg_word_len', 'avg_word_len_clean_sw',
       'RB', 'VBD','DT', 'JJ', 'NN', 'IN', 'VBZ', 'NNS', 'VBP', 'EX', 'VBG', 'VB', 'PRP',
       'CD', 'CC', 'MD', 'RBR', 'WRB', 'WP', 'VBN', 'RP', 'PRP$', '$', 'TO',
       'NNP', 'WDT', 'PDT', 'JJS', 'JJR', 'FW', 'RBS', 'X', 'UH']]

drop = []
for i in list(data_test.columns): 
  if i not in list(data_train.columns): 
    drop.append(i)

data_test.drop(columns= drop, inplace=True)

drop = []
for i in list(data_train.columns): 
  if i not in list(data_test.columns) and i != 'target':
    drop.append(i)

data_train.drop(columns=drop, inplace=True)

X_train, y_train = data_train.iloc[:,1:], data_train['target']
X_test = data_test

X_train.fillna(value=0, inplace = True)
X_test.fillna(value=0, inplace = True)


# XGBoost Classifier test

In [4]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

## Hiper-params

* **learning_rate:** tasa de aprendizaje
* **max_depth:** máxima profundidad de cada árbol
* **subsample:** porcentaje de muestras usadas para cada árbol (valor muy bajo, posible underfitting)
* **colsample_bytree:** porcentaje de features usadas para cada árbol (valores muy alto, posible overfitting)
* **n_estimators:** cantidad de árboles a construir.
* **objective:** función de error a utilizar (algunas: reg:squarederror para regresión, reg:logistic o binary:logistic para clasificación)

Parámetros de regularización:

* **gamma:** umbral para hacer split basado en la reducción de error de hacer el nuevo split.
* **alpha:** regularización para los pesos de las hojas. Un valor más alto genera una mayor regularización.
* **lambda:** similar alpha pero para la sintonia fina.

## K-fold Cross-Validation

Se usa el método **cv**

Hiper-parámetros:

* **nfolds:** K del k-fold
* **num_boost_round:** cantidad de árboles a contruir (n_estimators)
* **metrics:** la métrica de evaluación a utilizar
* **as_pandas:** si los resultados lo devuelve en un DataFrame de pandas
* **early_stopping_rounds:** terminar antes si la métrica no mejora luego de una cantidad de pasadas
* **seed:** semilla para poder reproducir los resultados


### Data to DMatrix object

In [ ]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

## RondomizedSearch CV with XGBoost

In [ ]:
 from scipy import stats
 from scipy.stats import randint
 from sklearn.model_selection import RandomizedSearchCV
 from sklearn.metrics import accuracy_score,roc_auc_score

In [ ]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(70, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.3, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.3, 0.9),
             }

clf_random = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 100, scoring = 'roc_auc',\
                         verbose = 3, n_jobs = -1, return_train_score=True, cv=10)

clf_random.fit(X_train,y_train)

import winsound         # for sound  

winsound.Beep(440, 250)